# 5. Deployment (TDSP Step 4)

Pada tahap deployment, model yang sudah dilatih diintegrasikan ke dalam alur kerja yang bisa digunakan oleh aplikasi atau pengguna bisnis.

Tujuan utama tahap ini:
- Melatih kembali atau memuat model terbaik.
- Menyimpan model ke dalam file yang bisa digunakan ulang.
- Menyediakan fungsi sederhana untuk melakukan prediksi harga.
- Menyiapkan gambaran integrasi model ke layanan atau API.

Notebook ini akan:
1. Memuat dataset bersih.
2. Melatih satu model terbaik (sebagai contoh digunakan Random Forest).
3. Menyimpan model ke file dengan joblib.
4. Membuat fungsi `predict_price` yang menerima satu record mobil dan mengembalikan prediksi harga.


In [9]:
# 5.1 Import library dan load dataset bersih
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

import joblib
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Load dataset bersih
data_path_clean = '../Dataset/UsedCarsSA_Clean.csv'
df_clean = pd.read_csv(data_path_clean)

df_clean.head()

,Make,Type,Year,Origin,Color,Options,Engine_Size,Fuel_Type,Gear_Type,Mileage,Region,Price,Car_Age
0,Chrysler,C300,2018,Saudi,Black,Full,5.7,Gas,Automatic,103000,Riyadh,114000.0,7
1,Nissan,Sunny,2019,Saudi,Silver,Standard,1.5,Gas,Automatic,72418,Riyadh,27500.0,6
2,Hyundai,Elantra,2019,Saudi,Grey,Standard,1.6,Gas,Automatic,114154,Riyadh,43000.0,6
3,Hyundai,Elantra,2019,Saudi,Silver,Semi Full,2.0,Gas,Automatic,41912,Riyadh,59500.0,6
4,Honda,Accord,2018,Saudi,Navy,Full,1.5,Gas,Automatic,39000,Riyadh,72000.0,7


## 5.2 Menyiapkan Fitur, Target, dan Preprocessing

Kita ulangi langkah penting secara ringkas agar notebook ini dapat berdiri sendiri.

- Target: kolom `Price`.
- Fitur: semua kolom selain `Price`.
- Tambahkan fitur `Car_Age` jika belum tersedia.
- Siapkan pipeline preprocessing untuk fitur numerik dan kategorikal.


In [10]:
# Pastikan Car_Age tersedia
if 'Car_Age' not in df_clean.columns and 'Year' in df_clean.columns:
    df_clean['Car_Age'] = 2025 - df_clean['Year']

# Definisikan target dan fitur
target = 'Price'
X = df_clean.drop(columns=[target])
y = df_clean[target]

# Bagi data menjadi train dan test (untuk pemeriksaan singkat)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# Tentukan fitur numerik dan kategorikal
numeric_features = []
for col in ['Year', 'Engine_Size', 'Mileage', 'Car_Age']:
    if col in X_train.columns:
        numeric_features.append(col)
categorical_features = [col for col in X_train.columns if col not in numeric_features]

numeric_features, categorical_features[:10]

(['Year', 'Engine_Size', 'Mileage', 'Car_Age'],
 ['Make',
  'Type',
  'Origin',
  'Color',
  'Options',
  'Fuel_Type',
  'Gear_Type',
  'Region'])

In [11]:
# Pipeline preprocessing
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['Year', 'Engine_Size', 'Mileage', 'Car_Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Make', 'Type', 'Origin', 'Color', 'Options',
                                  'Fuel_Type', 'Gear_Type', 'Region'])])

## 5.3 Melatih Model yang Akan Dideploy

Sebagai contoh, kita gunakan `RandomForestRegressor` sebagai model final yang akan disimpan dan dideploy.

Pada praktik di dunia nyata, pemilihan model ini biasanya berdasarkan hasil evaluasi di tahap modeling (Section 4).

In [12]:
# Definisikan pipeline penuh (preprocessing + model)
deployment_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(
        n_estimators=300,
        random_state=42,
        n_jobs=-1
    ))
])

# Latih model
deployment_model.fit(X_train, y_train)

print('Model untuk deployment berhasil dilatih.')

Model untuk deployment berhasil dilatih.


## 5.4 Menyimpan Model ke File

Kita menyimpan model ke dalam file `.joblib` agar dapat digunakan kembali tanpa perlu melatih ulang setiap kali.


In [13]:
# Simpan model
model_path = 'used_car_price_model_rf.joblib'
joblib.dump(deployment_model, model_path, compress=('xz', 3))
print(f'Model disimpan ke: {model_path}')

Model disimpan ke: used_car_price_model_rf.joblib


## 5.5 Fungsi Prediksi `predict_price`

Sekarang kita definisikan fungsi pembungkus yang menerima satu record mobil dalam bentuk dictionary dan mengembalikan hasil prediksi harga.

Contoh struktur input:

```python
sample_car = {
    'Make': 'Hyundai',
    'Type': 'Accent',
    'Year': 2020,
    'Origin': 'Sau',
    'Gear_Type': 'Automatic',
    'Engine_Size': 1.6,
    'Fuel_Type': 'Petrol',
    'Region': 'Riyadh',
    'Mileage': 40000,
    'Options': 'Full option'
}
```

In [14]:
# Muat ulang model dari file
loaded_model = joblib.load(model_path)

def predict_price(single_record: dict) -> float:
    """Menerima satu record mobil dalam bentuk dictionary dan mengembalikan prediksi harga.

    Parameter
    ----------
    single_record : dict
        Contoh:
        {
            'Make': 'Hyundai',
            'Type': 'Accent',
            'Year': 2020,
            'Origin': 'Sau',
            'Gear_Type': 'Automatic',
            'Engine_Size': 1.6,
            'Fuel_Type': 'Petrol',
            'Region': 'Riyadh',
            'Mileage': 40000,
            'Options': 'Full option'
        }
    """
    df_input = pd.DataFrame([single_record])

    # Jika fitur Car_Age digunakan di training, hitung juga di sini
    if 'Year' in df_input.columns and 'Car_Age' in X_train.columns and 'Car_Age' not in df_input.columns:
        df_input['Car_Age'] = 2025 - df_input['Year']

    # Pastikan kolom di input sama seperti X_train (untuk fitur yang tidak ada, isi NaN)
    for col in X_train.columns:
        if col not in df_input.columns:
            df_input[col] = np.nan
    df_input = df_input[X_train.columns]

    prediction = loaded_model.predict(df_input)[0]
    return float(prediction)

print('Fungsi predict_price siap digunakan.')

Fungsi predict_price siap digunakan.


## 5.6 Contoh Penggunaan Fungsi Prediksi

Kita uji fungsi `predict_price` dengan sebuah contoh mobil yang disimulasikan.


In [15]:
sample_car = {
    'Make': 'Hyundai',
    'Type': 'Accent',
    'Year': 2020,
    'Origin': 'Sau',
    'Gear_Type': 'Automatic',
    'Engine_Size': 1.6,
    'Fuel_Type': 'Petrol',
    'Region': 'Riyadh',
    'Mileage': 40000,
    'Options': 'Full option'
}

predicted_price = predict_price(sample_car)
print(f'Prediksi harga: {predicted_price:,.0f} SAR')

Prediksi harga: 52,172 SAR


## 5.7 Gambaran Integrasi ke Aplikasi atau API

Beberapa opsi untuk mengintegrasikan model ini ke sistem nyata:

1. API berbasis web
   - Bungkus fungsi `predict_price` menggunakan framework seperti FastAPI atau Flask.
   - Aplikasi front end atau sistem lain dapat mengirim payload JSON berisi fitur mobil dan menerima respons berisi prediksi harga.

2. Batch scoring
   - Jalankan skrip Python terjadwal yang membaca batch data mobil dari database atau file.
   - Tulis hasil prediksi kembali ke tabel atau file terpisah untuk dianalisis oleh tim bisnis.

3. Integrasi langsung ke platform jual beli
   - Saat penjual mengisi form input mobil, panggil model secara real time.
   - Tampilkan rekomendasi harga dan rentang harga wajar berdasarkan output model.

Tahap berikutnya dalam TDSP adalah *Customer or Stakeholder Acceptance*, yaitu memvalidasi apakah hasil deployment ini sudah memenuhi kebutuhan bisnis dan mudah dioperasikan oleh tim terkait.
